In [13]:
%load_ext autoreload
%autoreload 2

In [1]:
from sentence_transformers import SentenceTransformer, util

In [3]:
import pandas as pd

# Course Data Loader & Preprocessing
Load all available course with metadata

In [9]:
course_raw_file = "data/course_raw.csv"

In [14]:
# convert csv file into dataframe
DataProcess = DataProcess()
df = DataProcess.load_data(course_raw_file)
df.head()

NameError: name 'DataProcess' is not defined

In [82]:
df.isna().sum()

,0
ID,0
COURSE_ID,0
COURSE_NAME,0
CREDITS,0
ELECTIVE,0
HASSTUDIED,0
COURSE_DIFFICULTY,0
PREREQUISITES,45
COREQUISITE,58
PREVIOUS,47


In [83]:
df = df.fillna(0)

# Features (User's Input):
completed courses, numbers of semesters, future goals, average difficulty of each semester

Future goals: case-extraction

In [84]:
user_input = "I want to make intelligent programs that understand images"

In [85]:
model = SentenceTransformer('all-MiniLM-L6-v2')
user_embedding = model.encode(user_input)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [86]:
course_description = pd.DataFrame(df['COURSE_DESCRIPTION'])
course_description.head()

,COURSE_DESCRIPTION
0,This course introduces the principles of deep ...
1,This course covers fundamental calculus concep...
2,An introduction to structured and procedural p...
3,"Fundamentals of mechanics including motion, fo..."
4,A beginner-level course in academic English li...


In [87]:
course_description = df["COURSE_DESCRIPTION"].astype(str).tolist()

In [88]:
print(course_description)

['This course introduces the principles of deep learning, including neural networks, convolutional neural networks (CNNs), recurrent networks, and autoencoders, with practical applications in image and language processing.', 'This course covers fundamental calculus concepts such as limits, derivatives, and integrals, with applications in engineering and science.', 'An introduction to structured and procedural programming using the C and C++ languages, covering syntax, control structures, functions, and file handling.', 'Fundamentals of mechanics including motion, forces, energy, momentum, and rotational dynamics, tailored for science and engineering students.', 'A beginner-level course in academic English listening skills, focusing on understanding lectures, note-taking, and recognizing main ideas and details.', 'This course develops basic academic writing skills, including paragraph structure, coherence, and grammar for clear written communication.', 'A foundational course introducing

In [89]:
flattened = [item[0] for item in course_description]
course_embeddings = model.encode(flattened)

course_embeddings.shape

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


(60, 384)

In [90]:
similarities = util.cos_sim(user_embedding, course_embeddings)
print(similarities)

tensor([[0.0558, 0.0558, 0.0517, 0.0575, 0.0517, 0.0558, 0.0517, 0.0517, 0.0558,
         0.0517, 0.0835, 0.0517, 0.0400, 0.0558, 0.1039, 0.1039, 0.0703, 0.0632,
         0.0517, 0.0632, 0.1039, 0.0575, 0.0517, 0.0334, 0.0517, 0.1039, 0.0575,
         0.0558, 0.1039, 0.0632, 0.0835, 0.1039, 0.1039, 0.0575, 0.0632, 0.0558,
         0.0835, 0.0575, 0.1039, 0.0517, 0.0575, 0.0517, 0.0558, 0.0835, 0.1039,
         0.0632, 0.1039, 0.0632, 0.0835, 0.1039, 0.0400, 0.0575, 0.0632, 0.1039,
         0.0575, 0.0517, 0.0517, 0.1039, 0.0835, 0.0558]])


In [93]:
top_indices = similarities.argsort(descending=True).tolist()[0]

for i in top_indices[:5]:
  print(df.iloc[i])

ID                                                                   15
COURSE_ID                                                       IT154IU
COURSE_NAME                                              Linear Algebra
CREDITS                                                               3
ELECTIVE                                                              0
HASSTUDIED                                                            0
COURSE_DIFFICULTY                                                 0.529
PREREQUISITES                                                   MA002IU
COREQUISITE                                                           0
PREVIOUS                                                              0
COURSE_DESCRIPTION    Continues Calculus 1 with techniques of integr...
Name: 14, dtype: object
ID                                                                   29
COURSE_ID                                                       IT150IU
COURSE_NAME                             

# Ontology

Ontology Base = the “conceptual set” (C1, C2, C3…)
👉 Think: concepts like Course, Student, Goal

Ontology Relations Library (Logic) = the “correlation set” (R1, R2, R3…)
👉 Think: relationships like hasGoal, requires, fitsGoal

Function Set = reasoning or inferencing rules
👉 Think: logic rules like
IF course fitsGoal AND student hasGoal THEN recommend(course)

Knowledge Base of Ontology = reusable knowledge based on domain-specific ontologies
👉 e.g., one for AI curriculum, one for Medicine, etc.

In [ ]:
from rdflib import Graph, Namespace, RDF, RDFS, URIRef, Literal

# Define Namespaces
EX = Namespace("http://example.org/")

# Create graph
g = Graph()
g.bind("ex", EX)

# --- Concepts (C1, C2, C3…) ---
g.add((EX.Student, RDF.type, RDFS.Class))
g.add((EX.Course, RDF.type, RDFS.Class))
g.add((EX.Goal, RDF.type, RDFS.Class))

# --- Relationships (R1, R2…) ---
g.add((EX.hasGoal, RDF.type, RDF.Property))
g.add((EX.hasGoal, RDFS.domain, EX.Student))
g.add((EX.hasGoal, RDFS.range, EX.Goal))

g.add((EX.fitsGoal, RDF.type, RDF.Property))
g.add((EX.fitsGoal, RDFS.domain, EX.Course))
g.add((EX.fitsGoal, RDFS.range, EX.Goal))

# --- Instances ---
g.add((EX.Alice, RDF.type, EX.Student))
g.add((EX.AI, RDF.type, EX.Goal))
g.add((EX.DeepLearning, RDF.type, EX.Course))

# --- Relations Between Instances ---
g.add((EX.Alice, EX.hasGoal, EX.AI))
g.add((EX.DeepLearning, EX.fitsGoal, EX.AI))

# --- Simple Inference (manual logic for now) ---
for student, _, goal in g.triples((None, EX.hasGoal, None)):
    for course, _, course_goal in g.triples((None, EX.fitsGoal, goal)):
        print(f"Recommend {course.split('/')[-1]} to {student.split('/')[-1]} because it fits goal: {goal.split('/')[-1]}")
